In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics
from pprint import pprint
from math import isnan

sp500_history = pd.read_csv('./sp500_history.csv')
sp500_history['date'] = pd.to_datetime(sp500_history['date']) 
sp500_history = sp500_history[['date', 'value', 'variable']]

start_date = datetime.strptime('2016-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['date'] > start_date) & (sp500_history['date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history.sort_values(['date', 'variable'], ascending=[True, True])

C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


date value        variable
261 2016-01-05  WLTW    added_ticker
260 2016-01-05  FOSL  removed_ticker
262 2016-01-19   EXR    added_ticker
263 2016-01-19    CB  removed_ticker
264 2016-02-01   FRT    added_ticker
..         ...   ...             ...
453 2019-12-23  ZBRA    added_ticker
454 2019-12-23   STE    added_ticker
449 2019-12-23   AMG  removed_ticker
451 2019-12-23   MAC  removed_ticker
452 2019-12-23  TRIP  removed_ticker

[195 rows x 3 columns]

In [2]:
calendar = pd.read_excel('./Nasdaq_Trading_Calendar.xlsx', sheet_name=None)
rebal_dates = []
for year in calendar:
    sheet = calendar[year]
    mask = sheet['S&P Indexes Rebalance S&P 500, S&P 400, and S&P 600'] == 1
    year_rebal_dates = sheet.loc[mask]
    for date in year_rebal_dates['Date'].values:
        rebal_dates.append(date)
pprint(rebal_dates)

[numpy.datetime64('2020-03-20T00:00:00.000000000'),
 numpy.datetime64('2020-06-19T00:00:00.000000000'),
 numpy.datetime64('2020-09-18T00:00:00.000000000'),
 numpy.datetime64('2020-12-18T00:00:00.000000000'),
 numpy.datetime64('2019-03-15T00:00:00.000000000'),
 numpy.datetime64('2019-06-21T00:00:00.000000000'),
 numpy.datetime64('2019-09-20T00:00:00.000000000'),
 numpy.datetime64('2019-12-20T00:00:00.000000000'),
 numpy.datetime64('2018-03-16T00:00:00.000000000'),
 numpy.datetime64('2018-06-15T00:00:00.000000000'),
 numpy.datetime64('2018-09-21T00:00:00.000000000'),
 numpy.datetime64('2018-12-21T00:00:00.000000000'),
 numpy.datetime64('2017-03-17T00:00:00.000000000'),
 numpy.datetime64('2017-06-16T00:00:00.000000000'),
 numpy.datetime64('2017-09-15T00:00:00.000000000'),
 numpy.datetime64('2017-12-15T00:00:00.000000000'),
 numpy.datetime64('2016-03-18T00:00:00.000000000'),
 numpy.datetime64('2016-06-17T00:00:00.000000000'),
 numpy.datetime64('2016-09-16T00:00:00.000000000'),
 numpy.datet

In [3]:
columns_to_create = ['original_date', 'total', 'up', 'count']

df_list = {}
df_list['regular_add'] = pd.DataFrame(columns=columns_to_create)
df_list['regular_delete'] = pd.DataFrame(columns=columns_to_create)
df_list['ad_hoc_add'] = pd.DataFrame(columns=columns_to_create)
df_list['ad_hoc_delete'] = pd.DataFrame(columns=columns_to_create)


In [4]:
MAX_DATE_EXTENSION = 10
days_bef_aft = 10 # the range of days before and after effective date to pull from Yahoo finance
last_date = sp500_history.iloc[0, sp500_history.columns.get_loc('date')]
last_changes = []
is_begin = True
sheet_name = ''
last_sheet_name = ''

for idx, row in sp500_history.iterrows():
    
    # End date is exclusive, so need to increase by 1
    prices = pdr.get_data_yahoo(row['value'], start=row['date'] - timedelta(days=days_bef_aft), end=row['date'] + timedelta(days=days_bef_aft + 1))
    if len(prices) == 0:
        continue
        
    # Find the adjusted effective date 
    # which is one trading day before the effective (market opening) date
    adj_date = row['date'] - timedelta(days=1)
    date_extension = 0
    while not adj_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        adj_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    adj_date_close = prices.loc[adj_date]['Close']
    if isnan(adj_date_close):
        continue
        
    # Find the entry date, which is one trading day before the adjusted effective date
    prev_date = adj_date - timedelta(days=1)
    date_extension = 0
    while not prev_date in prices.index and date_extension < MAX_DATE_EXTENSION:
        prev_date -= timedelta(days=1)
        date_extension += 1
    if date_extension >= MAX_DATE_EXTENSION:
        continue
    prev_close = prices.loc[prev_date]['Close']
    if isnan(prev_close):
        continue

    price_change = adj_date_close / prev_close - 1
    
    if adj_date in rebal_dates:
        if row['variable'] == 'added_ticker':
            sheet_name = 'regular_add'
        else:
            sheet_name = 'regular_delete'
            
    else:
        if row['variable'] == 'added_ticker':
            sheet_name = 'ad_hoc_add'
        else:
            sheet_name = 'ad_hoc_delete'

    # Calculate mean and all
    if not is_begin and (last_date != adj_date or last_sheet_name != sheet_name):
        count = len(last_changes)
        up = len(list(filter(lambda x: (x >= 0), last_changes))) 
        
        data = [up, count]

        df_list[last_sheet_name].loc[last_date, 'up':'count'] = data
        
        total = 0
        last_changes = []
    
    # Still in the same date
    else:
        if is_begin:
            total = 0
            is_begin = False
        else:
            total = last_row['total']
    
    total += price_change
    
    data = {'total': total, 'original_date': row['date']}
            
    if adj_date not in df_list[sheet_name].index:
        df_list[sheet_name] = df_list[sheet_name].append(pd.DataFrame(data, index = [pd.to_datetime(adj_date)]), ignore_index=False)
    else:
        df_list[sheet_name].loc[adj_date, 'total'] = total
                
    last_date = adj_date
    last_changes.append(price_change)
    last_sheet_name = sheet_name
    last_row = df_list[sheet_name].loc[last_date, :]

for sheet in df_list:
    df_list[sheet].index.names = ['date']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRCM: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PCP: Data doesn't exist for startDate = 1453392000, endDate = 1455206400
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WFM: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- Q: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***

In [5]:
'''
Calculate population statistics
'''
stats_df = pd.DataFrame(columns=['total', 'up', 'count', 'mean', 'std', 'max', 'min', 'sharpe', 'uppct'])

for sheet in df_list:
    sheet_sum = df_list[sheet].sum(axis=0)
    mean = sheet_sum['total'] / sheet_sum['count']
    if sheet_sum['count'] > 1:
        std = statistics.stdev(df_list[sheet]['total'])
    else:
        std = 0
    max_val = max(df_list[sheet]['total'])
    min_val = min(df_list[sheet]['total'])
    if std == 0:
        sharpe = mean / 0.00001 * np.sqrt(252)
    else:
        sharpe = mean / std * np.sqrt(252)
    uppct = sheet_sum['up'] / sheet_sum['count']
    data = {'total': sheet_sum['total'], 'up': sheet_sum['up'], 'count': sheet_sum['count'], 
            'mean': mean, 'std': std, 'max': max_val, 'min': min_val, 'sharpe': sharpe, 'uppct': uppct} 
    stats_df = stats_df.append(pd.DataFrame(data, index=[sheet]), ignore_index=False)
    

In [6]:
'''
Writing to excel
'''
writer = pd.ExcelWriter('sp500_analysis.xlsx', engine='xlsxwriter')
for sheet in df_list:
    df_list[sheet].to_excel(writer, sheet_name=sheet)
stats_df.to_excel(writer, sheet_name='population_stats')
writer.save()